# Start EVA Server

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/georgia-tech-db/eva/blob/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table>

### Launch EVA server

We use this notebook for launching the EVA server.

In [1]:
## Install EVA package if needed
#%pip install "evadb[udf]" --quiet

import os
import time
from psutil import process_iter
from signal import SIGTERM
import re

def shell(command):
    print(command)
    os.system(command)

def kill_server_with_pid(pid):
    for proc in process_iter():
        if proc.pid == pid:
            proc.send_signal(SIGTERM)

def stop_eva_server():
    # FIND eva PID
    pid = None
    try:
        with open('eva.log', 'r') as f:
            for line in f:
                if 'PID' in line:
                    pid = re.findall(r'\(\d+\)', line)[0]
                    pid = pid[1:-1]
    except FileNotFoundError:
        pass

    if pid is not None:
        kill_server_with_pid(pid)

def launch_eva_server():
    # Stop EVA server if it is running
    stop_eva_server()

    os.environ['GPU_DEVICES'] = '0'

    # Start EVA server
    shell("nohup eva_server > eva.log 2>&1 &")

    try:
        with open('eva.log', 'r') as f:
            print(f.read())
    except FileNotFoundError:
        pass

    # Wait for server to start
    time.sleep(10)

def connect_to_server():
    from eva.server.db_api import connect
    %pip install nest_asyncio --quiet
    import nest_asyncio
    nest_asyncio.apply()

    # Connect client with server
    connection = connect(host = '127.0.0.1', port = 5432) 
    cursor = connection.cursor()

    return cursor

# Launch server
launch_eva_server()

[  -z "$(lsof -ti:5432)" ] || kill -9 $(lsof -ti:5432)
nohup eva_server > eva.log 2>&1 &
01-01-2023 15:00:28 INFO  [catalog_manager:catalog_manager.py:_bootstrap_catalog:0073] Bootstrapping catalog
01-01-2023 15:00:28 INFO  [base_model:base_model.py:init_db:0103] Creating tables
01-01-2023 15:00:28 INFO  [timer:timer.py:log_elapsed_time:0045] Query Compile Time: 0.2603 sec
01-01-2023 15:00:28 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF FastRCNNObjectDetector already exists, nothing added.
01-01-2023 15:00:28 INFO  [timer:timer.py:log_elapsed_time:0045] Query Compile Time: 0.0008 sec
01-01-2023 15:00:28 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF Array_Count already exists, nothing added.
01-01-2023 15:00:28 INFO  [timer:timer.py:log_elapsed_time:0045] Query Compile Time: 0.0007 sec
01-01-2023 15:00:28 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF Crop already exists, nothing added.
01-01-2023 15:00:28 INFO  [timer:timer.py